In [2]:
import streamlit as st
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
import os

# Page configuration
st.set_page_config(
    page_title="HealthGuardian AI",
    page_icon="🏥",
    layout="centered"
)

# Title and description
st.title("🏥 HealthGuardian AI")
st.markdown("Intelligent Medical Triage System")
st.markdown("---")

# Load or create training data
@st.cache_data
def load_data():
    data_path = "data/symptom_conditions.csv"

    # Create sample data if it doesn't exist
    sample_data = {
        'symptoms': [
            'fever headache runny_nose',
            'high_fever chills body_ache',
            'chest_pain shortness_breath',
            'abdominal_pain nausea vomiting',
            'dry_cough shortness_breath',
            'mild_headache',
            'skin_rashes fever',
            'sore_throat cough',
            'dizziness blurred_vision',
            'ear_pain fever'
        ],
        'condition': [
            'flu', 'flu', 'cardiac_emergency', 'gastroenteritis',
            'covid_suspicion', 'headache', 'dengue_suspicion', 'tonsillitis',
            'blood_pressure', 'otitis'
        ],
        'urgency': ['low', 'medium', 'high', 'medium', 'high',
                   'low', 'high', 'medium', 'high', 'medium'],
        'recommendations': [
            'rest hydration monitor_temperature',
            'rest hydration fever_medication consult_24h',
            'seek_immediate_care',
            'light_diet hydration consult_48h',
            'isolation covid_test urgent_consultation',
            'rest pain_reliever',
            'rest hydration seek_urgent_care',
            'gargle pain_reliever medical_consultation',
            'check_blood_pressure seek_urgent_care',
            'pain_reliever medical_consultation'
        ]
    }

    df = pd.DataFrame(sample_data)
    os.makedirs("data", exist_ok=True)
    df.to_csv(data_path, index=False)
    return df

# Train model
@st.cache_resource
def train_model(df):
    vectorizer = TfidfVectorizer(max_features=100)
    X = vectorizer.fit_transform(df['symptoms'])
    y_condition = df['condition']
    y_urgency = df['urgency']

    model_condition = RandomForestClassifier(n_estimators=50, random_state=42)
    model_urgency = RandomForestClassifier(n_estimators=50, random_state=42)

    model_condition.fit(X, y_condition)
    model_urgency.fit(X, y_urgency)

    return vectorizer, model_condition, model_urgency

# Urgency mapping
urgency_info = {
    'low': {'color': '🟢', 'desc': 'LOW - Consultation in 48-72h'},
    'medium': {'color': '🟡', 'desc': 'MEDIUM - Consultation in 24-48h'},
    'high': {'color': '🔴', 'desc': 'HIGH - Seek immediate care'}
}

# Recommendations mapping
recommendation_map = {
    'rest': '• Get adequate rest',
    'hydration': '• Stay hydrated',
    'monitor_temperature': '• Monitor temperature every 6 hours',
    'fever_medication': '• Fever medication if necessary',
    'consult_24h': '• Schedule appointment within 24 hours',
    'consult_48h': '• Schedule appointment within 48 hours',
    'seek_immediate_care': '• 🚨 SEEK IMMEDIATE MEDICAL CARE',
    'isolation': '• Stay in isolation',
    'covid_test': '• Get tested for COVID-19',
    'pain_reliever': '• Pain reliever for discomfort',
    'light_diet': '• Maintain light diet',
    'gargle': '• Gargle with warm salt water',
    'check_blood_pressure': '• Check blood pressure',
    'medical_consultation': '• Consult doctor for evaluation',
    'urgent_consultation': '• Urgent medical consultation needed',
    'seek_urgent_care': '• Seek urgent medical care'
}

def format_recommendations(rec_text):
    recommendations = []
    for rec_key, rec_display in recommendation_map.items():
        if rec_key in rec_text:
            recommendations.append(rec_display)
    return recommendations

# Main interface
def main():
    df = load_data()
    vectorizer, model_condition, model_urgency = train_model(df)

    st.subheader("📋 Describe Your Symptoms")
    symptoms_text = st.text_area(
        "Example: fever, headache, runny nose, nausea...",
        height=100,
        placeholder="Describe all the symptoms you're experiencing..."
    )

    if st.button("🔍 Analyze Symptoms", type="primary"):
        if symptoms_text.strip():
            symptoms_vec = vectorizer.transform([symptoms_text])
            condition_pred = model_condition.predict(symptoms_vec)[0]
            urgency_pred = model_urgency.predict(symptoms_vec)[0]

            matched_row = df[df['condition'] == condition_pred].iloc[0]
            recommendations = format_recommendations(matched_row['recommendations'])

            st.markdown("---")
            st.subheader("📊 Triage Results")

            urgency_data = urgency_info[urgency_pred]
            st.markdown(f"### {urgency_data['color']} URGENCY LEVEL: {urgency_data['desc']}")
            st.markdown(f"### 🩺 POSSIBLE CONDITIONS: {condition_pred.replace('_', ' ').title()}")

            st.markdown("### 💡 RECOMMENDATIONS:")
            for rec in recommendations:
                st.markdown(rec)

            st.markdown("---")
            st.warning("""
            ⚠️ **IMPORTANT NOTICE:**
            This is a preliminary triage system and **DOES NOT REPLACE**
            evaluation by a healthcare professional.
            Always consult a doctor for accurate diagnosis.
            """)
        else:
            st.error("Please describe your symptoms for analysis.")

    st.markdown("---")
    st.markdown("### ℹ️ About the System")
    st.info("""
    HealthGuardian AI uses machine learning to:
    - Analyze symptom descriptions
    - Suggest possible conditions
    - Indicate urgency level
    - Provide preliminary guidance

    **Remember:** This tool is for initial guidance only.
    """)

if __name__ == "__main__":
    main()



ModuleNotFoundError: No module named 'streamlit'

In [3]:
!pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 81.2 MB/s eta 0:00:00


In [4]:
import streamlit as st
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
import os

# Page configuration
st.set_page_config(
    page_title="HealthGuardian AI",
    page_icon="🏥",
    layout="centered"
)

# Title and description
st.title("🏥 HealthGuardian AI")
st.markdown("Intelligent Medical Triage System")
st.markdown("---")

# Load or create training data
@st.cache_data
def load_data():
    data_path = "data/symptom_conditions.csv"

    # Create sample data if it doesn't exist
    sample_data = {
        'symptoms': [
            'fever headache runny_nose',
            'high_fever chills body_ache',
            'chest_pain shortness_breath',
            'abdominal_pain nausea vomiting',
            'dry_cough shortness_breath',
            'mild_headache',
            'skin_rashes fever',
            'sore_throat cough',
            'dizziness blurred_vision',
            'ear_pain fever'
        ],
        'condition': [
            'flu', 'flu', 'cardiac_emergency', 'gastroenteritis',
            'covid_suspicion', 'headache', 'dengue_suspicion', 'tonsillitis',
            'blood_pressure', 'otitis'
        ],
        'urgency': ['low', 'medium', 'high', 'medium', 'high',
                   'low', 'high', 'medium', 'high', 'medium'],
        'recommendations': [
            'rest hydration monitor_temperature',
            'rest hydration fever_medication consult_24h',
            'seek_immediate_care',
            'light_diet hydration consult_48h',
            'isolation covid_test urgent_consultation',
            'rest pain_reliever',
            'rest hydration seek_urgent_care',
            'gargle pain_reliever medical_consultation',
            'check_blood_pressure seek_urgent_care',
            'pain_reliever medical_consultation'
        ]
    }

    df = pd.DataFrame(sample_data)
    os.makedirs("data", exist_ok=True)
    df.to_csv(data_path, index=False)
    return df

# Train model
@st.cache_resource
def train_model(df):
    vectorizer = TfidfVectorizer(max_features=100)
    X = vectorizer.fit_transform(df['symptoms'])
    y_condition = df['condition']
    y_urgency = df['urgency']

    model_condition = RandomForestClassifier(n_estimators=50, random_state=42)
    model_urgency = RandomForestClassifier(n_estimators=50, random_state=42)

    model_condition.fit(X, y_condition)
    model_urgency.fit(X, y_urgency)

    return vectorizer, model_condition, model_urgency

# Urgency mapping
urgency_info = {
    'low': {'color': '🟢', 'desc': 'LOW - Consultation in 48-72h'},
    'medium': {'color': '🟡', 'desc': 'MEDIUM - Consultation in 24-48h'},
    'high': {'color': '🔴', 'desc': 'HIGH - Seek immediate care'}
}

# Recommendations mapping
recommendation_map = {
    'rest': '• Get adequate rest',
    'hydration': '• Stay hydrated',
    'monitor_temperature': '• Monitor temperature every 6 hours',
    'fever_medication': '• Fever medication if necessary',
    'consult_24h': '• Schedule appointment within 24 hours',
    'consult_48h': '• Schedule appointment within 48 hours',
    'seek_immediate_care': '• 🚨 SEEK IMMEDIATE MEDICAL CARE',
    'isolation': '• Stay in isolation',
    'covid_test': '• Get tested for COVID-19',
    'pain_reliever': '• Pain reliever for discomfort',
    'light_diet': '• Maintain light diet',
    'gargle': '• Gargle with warm salt water',
    'check_blood_pressure': '• Check blood pressure',
    'medical_consultation': '• Consult doctor for evaluation',
    'urgent_consultation': '• Urgent medical consultation needed',
    'seek_urgent_care': '• Seek urgent medical care'
}

def format_recommendations(rec_text):
    recommendations = []
    for rec_key, rec_display in recommendation_map.items():
        if rec_key in rec_text:
            recommendations.append(rec_display)
    return recommendations

# Main interface
def main():
    df = load_data()
    vectorizer, model_condition, model_urgency = train_model(df)

    st.subheader("📋 Describe Your Symptoms")
    symptoms_text = st.text_area(
        "Example: fever, headache, runny nose, nausea...",
        height=100,
        placeholder="Describe all the symptoms you're experiencing..."
    )

    if st.button("🔍 Analyze Symptoms", type="primary"):
        if symptoms_text.strip():
            symptoms_vec = vectorizer.transform([symptoms_text])
            condition_pred = model_condition.predict(symptoms_vec)[0]
            urgency_pred = model_urgency.predict(symptoms_vec)[0]

            matched_row = df[df['condition'] == condition_pred].iloc[0]
            recommendations = format_recommendations(matched_row['recommendations'])

            st.markdown("---")
            st.subheader("📊 Triage Results")

            urgency_data = urgency_info[urgency_pred]
            st.markdown(f"### {urgency_data['color']} URGENCY LEVEL: {urgency_data['desc']}")
            st.markdown(f"### 🩺 POSSIBLE CONDITIONS: {condition_pred.replace('_', ' ').title()}")

            st.markdown("### 💡 RECOMMENDATIONS:")
            for rec in recommendations:
                st.markdown(rec)

            st.markdown("---")
            st.warning("""
            ⚠️ **IMPORTANT NOTICE:**
            This is a preliminary triage system and **DOES NOT REPLACE**
            evaluation by a healthcare professional.
            Always consult a doctor for accurate diagnosis.
            """)
        else:
            st.error("Please describe your symptoms for analysis.")

    st.markdown("---")
    st.markdown("### ℹ️ About the System")
    st.info("""
    HealthGuardian AI uses machine learning to:
    - Analyze symptom descriptions
    - Suggest possible conditions
    - Indicate urgency level
    - Provide preliminary guidance

    **Remember:** This tool is for initial guidance only.
    """)

if __name__ == "__main__":
    main()

2025-10-25 12:02:49.920 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-25 12:02:49.923 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-25 12:02:50.070 
  command:

    streamlit run /usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-10-25 12:02:50.070 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-25 12:02:50.071 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-25 12:02:50.072 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-25 12:02:50.073 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when runn